# "Классификация договора в зависимости от предмета по второму уровню ОКПД2" ≡ "Извлечение содержания предмета сделки из текстов документов"

### Подготовка. Введение.
1. Поскольку коммерческие данные получить быстро не удалось, было принято решение использовать публичные данные, доступные на [госзакупках](https://zakupki.gov.ru/epz/main/public/home.html).
2. В связи с таким изменением подхода появилась необходимость переформулирования задачи, теперь она сформулирована так: "Классификация договора в зависимости от предмета по второму уровню ОКПД2". Классификатор ОКПД2 можно получить, например, [здесь](https://www.consultant.ru/document/cons_doc_LAW_163703/). Таким образом, цель изменилась до: по разделу предмета договора вернуть второй уровень кода ОКПД2.
Важное замечание: поскольку предметов договора в данном случае может быть много, то мы имеем дело с многоклассовой классификацией.
3. Датасет состоит из 199913 записей в формате JSON, содержащие три поля "regNum" - реестровый номер контракта, "contractSubjectFull" - полный текст предмета договора, "OKPD2_codes" - набор ОКПД2 кодов из договора. Датасет расположени по [адресу](https://www.kaggle.com/datasets/aldarovalexander/contract). Сам датасет был получен путем парсинга части исполненных договоров за 2022 год, которые бы имели формат верный DOCX и разбора формализованной части договоров. Часть процесса получения, например, из графического представления договора верного текста предмета в данном исследовании опущена, поскольку для целей самого исследования не представляет интереса.
4. Блокнот доступен в [колабе](https://colab.research.google.com/github/aldarovav/YearProject/blob/main/analysis.ipynb)


### Импорт библиотек и настройка

In [1]:
!pip install pymorphy3
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import os
import io
import requests
import json
import kagglehub
import pymorphy3
import nltk
import matplotlib.pyplot as plt
import tqdm

from io import StringIO
from collections import Counter

# Настройка отображения
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Библиотеки загружены")
print("Текущая рабочая директория:", os.getcwd()) #Необходимо для избежания путаницы между средами выполнения

### Разведочный анализ данных

In [ ]:
from kagglehub import KaggleDatasetAdapter
import pandas as pd

# Указываем конкретный файл для загрузки
file_path = "contracts_dataset_unique.json"

# Сначала получим путь к файлу
dataset_path = kagglehub.dataset_download("aldarovalexander/contract")
full_file_path = f"{dataset_path}/{file_path}"

# Загрузим JSON с указанием dtype - это необходимо, поскольку иначе read_json воспринимает regNum как число и обрезает его, что создает неуникальные записи
df = pd.read_json(full_file_path, dtype={'regNum': str})


In [ ]:

# Посмотрим первые записи
print("Первые 5 записей:", df.head(3))

In [ ]:
df.describe(include='all')

In [ ]:
# Первичный анализ
print(f"Размер датасета: {df.shape}")
print(f"Колонки: {list(df.columns)}")
print("\nТипы данных:")
print(df.dtypes)


In [ ]:
# Более детальный анализ
print(f"Общее количество записей: {len(df)}")
print(f"Количество уникальных номеров контрактов: {df['regNum'].nunique()}")
print(f"Неуникальные номера контрактов: {df[df.duplicated('regNum', keep=False)]}")


Очень хороший результат - все записи уникальны

In [ ]:
# Анализ ОКПД кодов (списков)
all_okpd2_codes = df['OKPD2_codes'].explode() # Развернем списки ОКПД2 кодов для анализа
print(f"Всего ОКПД2 кодов (с повторениями): {len(all_okpd2_codes)}")
print(f"Уникальных ОКПД2 кодов: {all_okpd2_codes.nunique()}")
top_10_okpd2 = all_okpd2_codes.value_counts().head(10)
print(f"Топ-10 самых частых ОКПД2 кодов: {top_10_okpd2}")


In [ ]:

# Анализ структуры попадания кодов ОКПД2 в договоры
okpd2_counts = df['OKPD2_codes'].str.len()
print(f"Количество контрактов с одним OKPD2 кодом: {(okpd2_counts == 1).sum()}")
print(f"Количество контрактов с несколькими OKPD2 кодами: {(okpd2_counts > 1).sum()}")
print(f"Максимальное количество OKPD2 кодов в одном контракте: {okpd2_counts.max()}")


#### Анализ классов

In [ ]:
# Анализ классов
combinations = df['OKPD2_codes'].apply(tuple).value_counts()

print("Топ-10 наборов ОКПД2:")
print(combinations.head(10))

print(f"\nВсего уникальных комбинаций: {len(combinations)}")
print(f"Записей с одним кодом: {(df['OKPD2_codes'].apply(len) == 1).sum()}")
print(f"Записей с несколькими кодами: {(df['OKPD2_codes'].apply(len) > 1).sum()}")

In [ ]:
# Считаем частоту каждого уникального набора ОКПД2
combinations = df['OKPD2_codes'].apply(tuple).value_counts()

print("📊 РАСПРЕДЕЛЕНИЕ НАБОРОВ ОКПД2")
print(f"Уникальных наборов: {len(combinations)}")
print(f"Всего договоров: {len(df)}")

# Топ-10 наборов
print("\nТоп-10 наборов:")
for i, (combo, count) in enumerate(combinations.head(10).items(), 1):
    print(f"{i}. {combo}: {count} договоров")

# График

combinations.plot(kind='line')
plt.title('Распределение наборов ОКПД2 по количеству договоров')
plt.ylabel('Количество договоров')
plt.yscale('log')  # Логарифмическая шкала по Y
plt.xticks([])  # Убираем подписи по оси X


Видим, что распределение классов даже на логарифмической шкале требует логарифмизации, что  указывает на неоднородность классов, построим кумулятивное распределение по классам, чтобы увидеть какая часть классов заметает какой объём договоров

In [ ]:
# Кумулятивное распределение
cumulative = combinations.cumsum()
total = len(df)

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumulative) + 1), cumulative.values)

# Ограничиваем ось абсцисс от 1 до 50
plt.xlim(1, 50)

# Добавляем линии процентов
percent_lines = [0.25, 0.5, 0.8, 0.9]
for p in percent_lines:
    value = total * p
    plt.axhline(y=value, color='red', linestyle='--', alpha=0.7)
    plt.text(0, value, f'{p*100:.0f}%', va='bottom', ha='left')

plt.title('Кумулятивное распределение наборов ОКПД2')
plt.ylabel('Кумулятивное количество договоров')
plt.xlabel('Количество классов (нарастающим итогом)')
plt.grid(True)
plt.tight_layout()
plt.show()

Вот теперь очень хорошо видно, чтобы получить 75% попадание, достаточно верно определить один из 20 крупнейших классов

#### Анализ текстов

In [ ]:
# Анализ текстовых данных
print(f"Средняя длина contractSubjectFull: {df['contractSubjectFull'].str.len().mean():.0f} символов")
print(f"Максимальная длина contractSubjectFull: {df['contractSubjectFull'].str.len().max()} символов")
print(f"Минимальная длина contractSubjectFull: {df['contractSubjectFull'].str.len().min()} символов")

min_length = df['contractSubjectFull'].str.len().min()
min_length_samples = df[df['contractSubjectFull'].str.len() == min_length].head(10)

print(f"\n10 примеров с минимальной длиной ({min_length} символов):")
for i, row in min_length_samples.iterrows():
    print(f"{i+1}. regNum: {row['regNum']}")
    print(f"   contractSubjectFull: '{row['contractSubjectFull']}'")
    print()

Очевидно, что есть некоторые проблемы с заполнением предмета договора. Есть основание считать, что менее 100 символов являются ошибками реализации выборки, проверим эту гипотезу

In [ ]:
# Выбор всех записей с длиной contractSubjectFull менее 100 символов
short_texts = df[df['contractSubjectFull'].str.len() < 100]

print(f"Найдено записей с длиной текста менее 100 символов: {len(short_texts)}")
print(f"Это составляет {len(short_texts) / len(df) * 100:.1f}% от общего количества записей")

# Вывод всех найденных записей
print("\nВсе записи с длиной contractSubjectFull < 100 символов:")
for i, (index, row) in enumerate(short_texts.iterrows(), 1):
    print(f"{i}. Длина: {len(row['contractSubjectFull'])} символов")
    print(f"   Текст: '{row['contractSubjectFull']}'")
    
    # Если есть другие столбцы, выводим их тоже
    other_columns = [col for col in df.columns if col != 'contractSubjectFull']
    for col in other_columns:
        print(f"   {col}: {row[col]}")
    print("-" * 50)

Действительно обрезанные записи, считаем их ошибочными и удалим из датасета

In [ ]:
# Находим индексы записей для удаления
indices_to_drop = df[df['contractSubjectFull'].str.len() < 100].index

# Удаляем записи
df = df.drop(indices_to_drop)

print(f"Удалено {len(indices_to_drop)} записей")
print(f"Осталось {len(df)} записей")

#### Анализ кодов ОКПД2

In [ ]:
# Проверим что идентификатор договора везде заполнен верно
print("Длина regNum:")
print(f"  Минимальная: {df['regNum'].str.len().min()}")
print(f"  Максимальная: {df['regNum'].str.len().max()}")
print(f"  Уникальные длины: {sorted(df['regNum'].str.len().unique())}")


In [ ]:
# Анализ распределения количества кодов на контракт

count_distribution = okpd2_counts.value_counts().sort_index()
for count, freq in count_distribution.items():
    print(f"  {count} ОКПД2: {freq} контрактов ({freq/len(df)*100:.1f}%)")

In [ ]:
# Строим график распределения частот
all_okpd2_freq = all_okpd2_codes.value_counts()  # Это сет с частотами для каждого уникального кода
plt.figure(figsize=(12, 6))
plt.plot(range(len(all_okpd2_freq)), all_okpd2_freq.values, linewidth=1)
plt.title('Распределение частот ОКПД2 кодов')
plt.xlabel('Ранг ОКПД2 кода')
plt.ylabel('Частота')
plt.grid(True)
plt.yscale('log')  # Логарифмическая шкала для лучшей визуализации
plt.show()


In [ ]:

# Дополнительный анализ распределения
print(f"Медианная частота: {all_okpd2_freq.median()}")
print(f"Средняя частота: {all_okpd2_freq.mean():.2f}")
print(f"Максимальная частота: {all_okpd2_freq.max()}")
print(f"Минимальная частота: {all_okpd2_freq.min()}")
print(f"Стандартное отклонение: {all_okpd2_freq.std():.2f}")


In [ ]:
# Анализ долей
total_mentions = all_okpd2_freq.sum()
top_10_percent_share = all_okpd2_freq.head(int(len(all_okpd2_freq) * 0.1)).sum() / total_mentions * 100
top_20_percent_share = all_okpd2_freq.head(int(len(all_okpd2_freq) * 0.2)).sum() / total_mentions * 100
top_25_percent_share = all_okpd2_freq.head(int(len(all_okpd2_freq) * 0.25)).sum() / total_mentions * 100

print(f"На 10% самых частых кодов приходится {top_10_percent_share:.1f}% всех упоминаний")
print(f"На 20% самых частых кодов приходится {top_20_percent_share:.1f}% всех упоминаний")
print(f"На 25% самых частых кодов приходится {top_25_percent_share:.1f}% всех упоминаний")

In [ ]:
# Кумулятивное распределение
cumulative_share = all_okpd2_freq.cumsum() / total_mentions * 100
plt.plot(range(len(all_okpd2_freq)), cumulative_share, linewidth=2, color='red')
plt.title('Кумулятивное распределение OKPD2 кодов')
plt.xlabel('Количество уникальных OKPD2 кодов')
plt.ylabel('Накопленная доля (%)')
plt.grid(True, alpha=0.3)

# Добавляем ориентиры
plt.axhline(y=80, color='gray', linestyle='--', alpha=0.7, label='80% всех упоминаний')
plt.axvline(x=len(all_okpd2_freq) * 0.2, color='gray', linestyle='--', alpha=0.7)
plt.legend()

plt.tight_layout()
plt.show()


В целом картина по распределению кодов и классов по заметаемым долям числа всех договоров почти совпадают, этого следовало ожидать, поскольку самые крупные классы содержат по одному коду ОКПД2

## Предобработка текста

In [ ]:
# Анализ текстов
print("Длина текстов после очистки пропущенных значений:")
print(df['contractSubjectFull'].str.len().describe())

In [ ]:
from tqdm import tqdm
#Функция очистки текста от мусора
def clean_text(text):
    if pd.isna(text) or text == '':
        return ""

    prev_text = text
    while True:
        text = text.lower()
        
        # Удаление приложений, ссылок, спецификаций
        text = re.sub(r'\(?(приложение|прил\.?|спецификаци[яией]|далее)\s*№?\s*\d*\)?', ' ', text, flags=re.IGNORECASE)
        
        # Удаление номеров контрактов и идентификаторов
        text = re.sub(r'(контракт|договор)\s*№?\s*\d+[/\-\\]\d+', ' ', text, flags=re.IGNORECASE)
        text = re.sub(r'идентификационный\s+код\s+закупки[:\-\s]*[\d\w]+', ' ', text, flags=re.IGNORECASE)
        
        # Удаление дат
        text = re.sub(r'\d{1,2}\s*[\.\/\\]\s*\d{1,2}\s*[\.\/\\]\s*\d{2,4}', ' ', text)
        text = re.sub(r'"\d{1,2}"\s*\w+\s*\d{4}\s*года?', ' ', text)
        
        # Удаление символа № и номеров
        text = re.sub(r'№\s*\d*', ' ', text)
        text = re.sub(r'\s№\s|^№\s|\s№$', ' ', text)
        
        # Удаление нумерации пунктов (1.1., 1.2. и т.д.)
        text = re.sub(r'\b\d+\.\d+\.', ' ', text)
        text = re.sub(r'^\d+\.', ' ', text)  # В начале строки
        
        # Удаление одиночных цифр
        text = re.sub(r'\s\d{1,2}\s', ' ', text)
        
        # Удаление лишних символов (сохраняем только буквы, цифры и пробелы)
        text = re.sub(r'[^a-zа-я0-9\s]', ' ', text)
        
        # Удаление множественных пробелов
        text = re.sub(r'\s+', ' ', text)
        
        text = text.strip()
        
        # Если текст перестал меняться - выходим
        if text == prev_text:
            break
        prev_text = text
    
    return text

tqdm.pandas(desc="Очистка текста")
df['text_clean'] = df['contractSubjectFull'].progress_apply(clean_text)

In [ ]:
a = 550
print("Пример очистки текста:")
print(df['regNum'].iloc[a])
print("ДО:", df['contractSubjectFull'].iloc[a][:200])
print("ПОСЛЕ:", df['text_clean'].iloc[a][:200])

In [ ]:
# Удаление стоп-слов
from nltk.corpus import stopwords

# Скачивание стоп-слов
nltk.download('stopwords')
russian_stopwords = stopwords.words('russian')

# Доменные стоп-слова
domain_stopwords = [
    'контракт', 'договор', 'приложение', 'пункт', 'статья', 'далее', 
    'согласно', 'также', 'например', 'иной', 'другой', 'обязан', 
    'обязана', 'обязаны', 'обязано', 'условие', 'следующий', 'документ', 
    'соглашение', 'протокол', 'решение', 'акт', 'отчет', 'настоящий',
    'настоящего', 'настоящем', 'настоящему']

# Добавляем предлоги и союзы
prepositions = ['по', 'в', 'и', 'с', 'у', 'на', 'за', 'к', 'о', 'от', 'до', 'из', 'без']
conjunctions = ['а', 'но', 'или', 'то', 'как', 'что', 'чтобы', 'if', 'and', 'or']

custom_stopwords = set(russian_stopwords + domain_stopwords + prepositions + conjunctions)
print(custom_stopwords)

#Функция удаления стоп-слов
def remove_stopwords(tokens):
    return [token for token in tokens if token.lower() not in custom_stopwords]


In [ ]:
# Создаем анализатор pymorphy3
morph = pymorphy3.MorphAnalyzer()

#Функция лемматизации текста
def lemmatize_tokens(tokens):
    lemmatized_tokens = []
    for token in tokens:
        try:
            parsed = morph.parse(token)[0]
            lemma = parsed.normal_form
            lemmatized_tokens.append(lemma)
        except Exception:
            lemmatized_tokens.append(token)
    return lemmatized_tokens

In [ ]:
#Функция токенизации текста 
def tokenize_text(text):
    return text.split()

In [ ]:
# Функция основной обработки текста
def advanced_text_processing(text):
    # Проверка на непустоту
    if not isinstance(text, str) or not text.strip():
        return ""
    
    # Очистка текста от мусора
    cleaned_text = clean_text(text)
    
    # Токенизация текста
    tokens = tokenize_text(cleaned_text)
    
    # Удаление стоп-слов (первый проход)
    tokens = remove_stopwords(tokens)
    
    # Лемматизация токенов
    lemmatized_tokens = lemmatize_tokens(tokens)
    
    # Удаление стоп-слов после лемматизации (второй проход)
    final_tokens = remove_stopwords(lemmatized_tokens)
    
    return ' '.join(final_tokens)

In [ ]:
from tqdm import tqdm

# Применяем ко всему датафрейму
print("Начата обработка всего датасета...")

def sequential_text_processing(series):

    texts = series.tolist()
    total_texts = len(texts)
   
    results = []
    
    # Обрабатываем каждый текст с прогресс-баром
    for text in tqdm(texts, total=total_texts, desc="Обработка текстов", unit="текст"):
        result = advanced_text_processing(text)
        results.append(result)
    
    return results

# Использование:
print("=== Последовательная обработка ===")
df['text_processed'] = sequential_text_processing(df['contractSubjectFull'])

print("Обработка завершена!")

In [ ]:
a = 550
print("Пример очистки текста:")
print(df['regNum'].iloc[a])
print("ДО:", df['contractSubjectFull'].iloc[a][:200])
print("ПОСЛЕ:", df['text_clean'].iloc[a][:200])
print("ПОСЛЕ:", df['text_processed'].iloc[a][:200])

In [ ]:
# Статистика обработки
print(f"Общее количество записей: {len(df)}")
print(f"Средняя длина исходного текста: {df['contractSubjectFull'].str.len().mean():.0f} символов")
print(f"Средняя длина обработанного текста: {df['text_processed'].str.len().mean():.0f} символов")
print(f"Сжатие текста: {(1 - df['text_processed'].str.len().mean() / df['contractSubjectFull'].str.len().mean()) * 100:.1f}%")

In [ ]:
# Показываем примеры обработки
for i in range(min(5, len(df))):
    print(f"\nПример {i+1}:")
    original = df['contractSubjectFull'].iloc[i]
    processed = df['text_processed'].iloc[i]
    
    print("ДО:", original[:150] + "..." if len(original) > 150 else original)
    print("ПОСЛЕ:", processed[:150] + "..." if len(processed) > 150 else processed)
    print("-" * 80)

In [ ]:
# Анализ эффективности обработки
empty_processed = df['text_processed'].str.strip().eq('').sum()
print(f"Пустых текстов после обработки: {empty_processed}")
print(f"Уникальных обработанных текстов: {df['text_processed'].nunique()}")

# Топ-10 самых частых слов после обработки
from collections import Counter
all_words = ' '.join(df['text_processed'].dropna()).split()
word_freq = Counter(all_words)
print(f"\nТоп-10 самых частых слов после обработки:")
for word, freq in word_freq.most_common(10):
    print(f"  {word}: {freq}")

Топ-10 самых частых слов после обработки:
  обязуется: 340701
  №: 281243
  (далее: 235462
  соответствии: 215894
  (Приложение: 205405
  1: 202434
  -: 198649
  Товара: 190669
  Поставщик: 183092
  (приложение: 178492

Такое распределение указывает на то, что можно подрезать скобки, знаки номеров, слова: обязуется, далее, приложение

### Разметка данных

In [ ]:
print(f"Всего записей: {len(df)}")
print(f"Записей с OKPD2: {df['OKPD2_codes'].notna().sum()}")
print(f"Процент размеченных данных: {df['OKPD2_codes'].notna().mean():.2%}")

# Анализ структуры OKPD2 кодов
df['okpd_count'] = df['OKPD2_codes'].apply(lambda x: len(x) if x else 0)
print("\nРаспределение количества кодов OKPD2 на запись:")
print(df['okpd_count'].value_counts().sort_index())

In [ ]:
# Извлечение и нормализация целевых меток
def extract_okpd_labels(okpd_codes):
    """Извлекает все коды ОКПД2 для классификации"""
    if not okpd_codes or len(okpd_codes) == 0:
        return []
    
    # Извлекаем все коды и нормализуем их до 2-значного уровня
    normalized_codes = []
    for code in okpd_codes:
        if len(code) >= 2:
            normalized_codes.append(code[:2])  # Первые 2 цифры - раздел
        else:
            normalized_codes.append(code)
    
    return normalized_codes

# Создание бинарной матрицы для мульти-лейбл классификации
from sklearn.preprocessing import MultiLabelBinarizer

# Создаем целевую переменную - список всех кодов ОКПД2
df['target'] = df['OKPD2_codes'].apply(extract_okpd_labels)

print("Распределение количества меток на запись:")
print(df['target'].apply(len).value_counts().sort_index())

print("\nОбщая статистика по меткам:")
print(f"Всего записей: {len(df)}")
print(f"Записей без меток: {df['target'].apply(len).eq(0).sum()}")

# Анализ меток
all_labels = [label for sublist in df['target'] for label in sublist]
print(f"Всего меток (с повторениями): {len(all_labels)}")

print("\nУровень детализации меток:")
label_lengths = pd.Series([len(label) for label in all_labels])
print(label_lengths.value_counts().sort_index())

print("\nТоп-10 самых частых меток ОКПД2:")
from collections import Counter
label_counts = Counter(all_labels)
for label, count in label_counts.most_common(10):
    print(f"{label}: {count} раз")

print(f"\nВсего уникальных меток: {len(label_counts)}")

# Получаем все уникальные метки (теперь столбец 'target' существует)
all_labels = [label for sublist in df['target'] for label in sublist]
unique_labels = sorted(set(all_labels))

print(f"Уникальные метки для классификации: {unique_labels}")

# Создаем бинарную матрицу
mlb = MultiLabelBinarizer(classes=unique_labels)
target_matrix = mlb.fit_transform(df['target'])

# Создаем DataFrame с бинарными метками
target_df = pd.DataFrame(target_matrix, columns=mlb.classes_, index=df.index)

print(f"\nСоздано {len(unique_labels)} уникальных меток ОКПД2")
print(f"Размер матрицы целевых переменных: {target_matrix.shape}")
print(f"Пример бинарных меток для первых 5 записей:")
print(target_df.head())

In [ ]:
# Анализ качества разметки для мульти-лейбл классификации
# Смотрим примеры текстов для самых частых классов

# Получаем самые частые отдельные метки (не списки)
top_labels = label_counts.most_common(5)
sample_labels = [label for label, count in top_labels]

for class_label in sample_labels:
    # Ищем записи, которые содержат данную метку
    mask = df['target'].apply(lambda x: class_label in x)
    class_texts = df[mask]['text_clean'].head(2)
    
    print(f"\n--- Метка {class_label} ({mask.sum()} примеров) ---")
    for i, text in enumerate(class_texts):
        print(f"{i+1}. {text[:150]}...")
    print(f"Все метки для этой категории: {df[mask]['target'].iloc[0] if len(df[mask]) > 0 else 'нет примеров'}")

In [ ]:
# Обработка проблем разметки для мульти-лейбл классификации

# 1. Анализ записей без меток (пустые списки)
empty_targets = df['target'].apply(len) == 0
print(f"Записей без меток (пустые списки): {empty_targets.sum()}")

# 2. Анализ малочисленных отдельных меток
# Собираем статистику по всем отдельным меткам
all_labels = [label for sublist in df['target'] for label in sublist]
label_counts = Counter(all_labels)

# Находим малочисленные метки (менее 5 примеров)
small_labels = {label: count for label, count in label_counts.items() if count < 5}
print(f"Малочисленные отдельные метки (менее 5 примеров): {len(small_labels)}")

# 3. Анализ записей с редкими метками
records_with_small_labels = df['target'].apply(
    lambda labels: any(label in small_labels for label in labels)
).sum()
print(f"Записей, содержащих хотя бы одну малочисленную метку: {records_with_small_labels}")

# 4. Решение: несколько стратегий
print("\n=== СТРАТЕГИИ ОБРАБОТКИ ===")

# Стратегия 1: Удаление записей без меток
df_clean = df[~empty_targets].copy()
print(f"После удаления пустых меток: {len(df_clean)} записей")

# Стратегия 2: Удаление малочисленных меток из списков
def remove_small_labels(labels, small_labels_set):
    """Удаляет малочисленные метки из списка"""
    return [label for label in labels if label not in small_labels_set]

small_labels_set = set(small_labels.keys())
df_clean['target_clean'] = df_clean['target'].apply(
    lambda x: remove_small_labels(x, small_labels_set)
)

# Проверяем, не остались ли записи без меток после очистки
empty_after_clean = df_clean['target_clean'].apply(len) == 0
print(f"Записей без меток после очистки: {empty_after_clean.sum()}")

# Удаляем записи, которые остались без меток после очистки
df_final = df_clean[~empty_after_clean].copy()

print(f"Итоговый размер датасета: {len(df_final)} записей")

# Анализ результатов очистки
all_labels_final = [label for sublist in df_final['target_clean'] for label in sublist]
label_counts_final = Counter(all_labels_final)

print(f"Уникальных меток после очистки: {len(label_counts_final)}")
print(f"Минимальное количество примеров на метку: {min(label_counts_final.values())}")
print(f"Максимальное количество примеров на метку: {max(label_counts_final.values())}")

# Статистика по количеству меток на запись после очистки
labels_per_record_final = df_final['target_clean'].apply(len)
print(f"\nРаспределение количества меток на запись после очистки:")
print(labels_per_record_final.value_counts().sort_index())

### Анализ размеченных данных

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Статистика для мульти-лейбл классификации
plt.figure(figsize=(15, 10))

# 1. Топ-20 самых частых отдельных меток
plt.subplot(2, 2, 1)
top_labels = label_counts_final.most_common(20)
labels, counts = zip(*top_labels)
sns.barplot(x=list(counts), y=list(labels))
plt.title('Топ-20 самых частых меток ОКПД2')
plt.xlabel('Количество вхождений')

# 2. Распределение количества меток на запись
plt.subplot(2, 2, 2)
labels_per_record = df['target'].apply(len)
sns.histplot(labels_per_record, bins=30, discrete=True)
plt.title('Распределение количества меток на запись')
plt.xlabel('Количество меток')
plt.ylabel('Количество записей')

# 3. Распределение размеров классов (отдельных меток)
plt.subplot(2, 2, 3)
class_sizes = list(label_counts_final.values())
sns.histplot(class_sizes, bins=30)
plt.title('Распределение частот отдельных меток')
plt.xlabel('Частота метки')
plt.ylabel('Количество меток')

# 4. Кумулятивное покрытие данных
plt.subplot(2, 2, 4)
cumulative_counts = []
current_sum = 0
for count in sorted(class_sizes, reverse=True):
    current_sum += count
    cumulative_counts.append(current_sum)

total_labels = sum(class_sizes)
cumulative_percentage = [x / total_labels * 100 for x in cumulative_counts]

plt.plot(range(1, len(cumulative_percentage) + 1), cumulative_percentage)
plt.title('Кумулятивное покрытие данных метками')
plt.xlabel('Количество топовых меток')
plt.ylabel('Процент покрытия данных (%)')
plt.grid(True)

plt.tight_layout()
plt.show()

# Статистика по мульти-лейбл данным
print("=== СТАТИСТИКА МУЛЬТИ-ЛЕЙБЛ ДАННЫХ ===")
print(f"Всего записей: {len(df)}")
print(f"Всего уникальных меток: {len(label_counts_final)}")
print(f"Общее количество вхождений меток: {sum(label_counts_final.values())}")

print(f"\nРаспределение количества меток на запись:")
labels_count_stats = labels_per_record.value_counts().sort_index()
for count, freq in labels_count_stats.items():
    percentage = freq / len(df) * 100
    print(f"  {count} меток: {freq} записей ({percentage:.1f}%)")

print(f"\nСтатистика по отдельным меткам:")
print(f"Медианная частота метки: {pd.Series(class_sizes).median()}")
print(f"Минимальная частота: {min(class_sizes)}")
print(f"Максимальная частота: {max(class_sizes)}")
print(f"Средняя частота: {pd.Series(class_sizes).mean():.1f}")

# Анализ покрытия
print(f"\nАнализ покрытия меток:")
top_n = [5, 10, 20, 50]
for n in top_n:
    if n <= len(label_counts_final):
        coverage = sum([count for _, count in label_counts_final.most_common(n)]) / sum(label_counts_final.values()) * 100
        print(f"Топ-{n} меток покрывают {coverage:.1f}% данных")

In [ ]:
# Дополнительный анализ сбалансированности
plt.figure(figsize=(15, 5))

# 1. Логарифмическое распределение частот меток
plt.subplot(1, 3, 1)
class_sizes_series = pd.Series(class_sizes)
sns.histplot(np.log1p(class_sizes_series), bins=30)
plt.title('Логарифмическое распределение частот меток')
plt.xlabel('log(частота + 1)')
plt.ylabel('Количество меток')

# 2. Количество меток vs количество записей
plt.subplot(1, 3, 2)
labels_per_record.value_counts().sort_index().plot(kind='bar')
plt.title('Распределение записей по количеству меток')
plt.xlabel('Количество меток')
plt.ylabel('Количество записей')

# 3. Топ-30 меток (более детально)
plt.subplot(1, 3, 3)
top_30_labels = label_counts_final.most_common(30)
labels_30, counts_30 = zip(*top_30_labels)
sns.barplot(x=list(counts_30), y=list(labels_30))
plt.title('Топ-30 самых частых меток ОКПД2')
plt.xlabel('Количество вхождений')

plt.tight_layout()
plt.show()

# Анализ самых популярных комбинаций меток
print("\n=== АНАЛИЗ КОМБИНАЦИЙ МЕТОК ===")
from collections import Counter

# Анализируем комбинации меток (первые 2-3 самые частые)
label_combinations = df['target'].apply(tuple).value_counts().head(10)

print("Топ-10 самых частых комбинаций меток:")
for i, (combo, count) in enumerate(label_combinations.items(), 1):
    print(f"{i}. {list(combo)}: {count} записей ({count/len(df)*100:.1f}%)")

# Анализ сбалансированности
print(f"\nКоэффициент сбалансированности (min/max): {min(class_sizes)/max(class_sizes):.4f}")
print(f"Стандартное отклонение частот: {pd.Series(class_sizes).std():.1f}")

In [ ]:
# Анализ для подготовки к ML
print("\n=== АНАЛИЗ ДЛЯ ПОДГОТОВКИ К ML ===")

# Рекомендации по обработке несбалансированности
imbalance_ratio = max(class_sizes) / min(class_sizes)
print(f"Коэффициент несбалансированности: {imbalance_ratio:.1f}")

if imbalance_ratio > 100:
    print("⚠️  Высокая несбалансированность - рекомендуется:")
    print("   - Использовать взвешенные функции потерь")
    print("   - Применить oversampling/undersampling")
    print("   - Использовать метрики, устойчивые к несбалансированности")
elif imbalance_ratio > 10:
    print("⚠️  Умеренная несбалансированность - рекомендуется:")
    print("   - Взвешенные функции потерь")
    print("   - Стратифицированная выборка")
else:
    print("✓  Данные достаточно сбалансированы")

# Анализ мульти-лейбл характеристик
avg_labels_per_record = labels_per_record.mean()
print(f"\nСреднее количество меток на запись: {avg_labels_per_record:.2f}")

if avg_labels_per_record > 3:
    print("⚠️  Высокая мульти-лейблность - рекомендуется:")
    print("   - Binary Relevance подход")
    print("   - Метрики: precision@k, coverage")
elif avg_labels_per_record > 1.5:
    print("✓  Умеренная мульти-лейблность - подходят:")
    print("   - Classifier Chains")
    print("   - Label Powerset")
else:
    print("✓  Низкая мульти-лейблность - можно использовать:")
    print("   - Любые мульти-лейбл методы")

### Разделение на выборки

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Используем наш обработанный датасет
X = df['text_clean']  # или 'text_processed', если есть
y = df['target']  # список меток для каждой записи

# Для мульти-лейбл данных используем iterative_train_test_split из skmultilearn
# или создаем свою стратегию стратификации

print(f"Всего данных для разделения: {len(X)} записей")

# Способ 1: Простое случайное разделение (без стратификации)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

print(f"Обучающая выборка: {len(X_train)} записей ({len(X_train)/len(X):.1%})")
print(f"Валидационная выборка: {len(X_val)} записей ({len(X_val)/len(X):.1%})")
print(f"Тестовая выборка: {len(X_test)} записей ({len(X_test)/len(X):.1%})")

In [ ]:
print(X_train[1])
print(df['target'][1])

### Векторизация текста

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# TF-IDF с разными настройками
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),  # Учитываем отдельные слова и пары
    min_df=2,           # Игнорируем очень редкие слова
    max_df=0.9,         # Игнорируем очень частые слова
    stop_words=list(custom_stopwords)
)

# Bag-of-Words для сравнения
bow_vectorizer = CountVectorizer(
    max_features=8000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.9,
    stop_words=list(custom_stopwords)
)

print("Векторизаторы созданы")

In [ ]:
# TF-IDF
print("Векторизация TF-IDF...")
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Bag-of-Words
print("Векторизация Bag-of-Words...")
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_val_bow = bow_vectorizer.transform(X_val)
X_test_bow = bow_vectorizer.transform(X_test)

print(f"\nРазмерности матриц признаков:")
print(f"TF-IDF: {X_train_tfidf.shape}")
print(f"BOW: {X_train_bow.shape}")

# Сохранение processed данных
df_processed = pd.DataFrame({
    'text_processed': X,
    'target': y
})


In [ ]:
# Преобразовать в плотный формат (для просмотра)
dense_vector = X_train_tfidf[1].toarray()
print("Плотный вектор формы:", dense_vector.shape)
print("Ненулевые элементы:", np.count_nonzero(dense_vector))

# Посмотреть ненулевые элементы с словами
feature_names = tfidf_vectorizer.get_feature_names_out()
nonzero_indices = X_train_tfidf[1].nonzero()[1]  # индексы ненулевых элементов

print("\nСлова и их TF-IDF веса:")
for idx in nonzero_indices:
    word = feature_names[idx]
    weight = X_train_tfidf[1][0, idx]
    print(f"  '{word}': {weight}")

In [ ]:
# Анализ Bag-of-Words матриц
print("🔍 АНАЛИЗ BAG-OF-WORDS МАТРИЦ")
print("=" * 50)

# Основная информация
print("\n1. ОСНОВНАЯ ИНФОРМАЦИЯ:")
print(f"X_train_bow: {X_train_bow.shape} (документы x слова)")
print(f"X_val_bow: {X_val_bow.shape}")
print(f"X_test_bow: {X_test_bow.shape}")

# Анализ разреженности
def analyze_sparsity(matrix, name):
    total = matrix.shape[0] * matrix.shape[1]
    nonzero = matrix.nnz
    density = (nonzero / total) * 100
    print(f"\n{name}:")
    print(f"  Ненулевых: {nonzero:,} из {total:,}")
    print(f"  Плотность: {density:.4f}%")
    return density

train_density = analyze_sparsity(X_train_bow, "TRAIN")
analyze_sparsity(X_val_bow, "VALIDATION")
analyze_sparsity(X_test_bow, "TEST")

# Статистика по значениям
print(f"\n2. СТАТИСТИКА ПО ЗНАЧЕНИЯМ:")
print(f"Min: {X_train_bow.min()}, Max: {X_train_bow.max()}")
print(f"Среднее: {X_train_bow.mean():.3f}")
print(f"Медиана ненулевых: {np.median(X_train_bow.data):.1f}")

# Распределение слов по документам
train_nz_per_doc = X_train_bow.getnnz(axis=1)
val_nz_per_doc = X_val_bow.getnnz(axis=1)

print(f"\n3. СЛОВ В ДОКУМЕНТАХ:")
print(f"Train: min={train_nz_per_doc.min()}, max={train_nz_per_doc.max()}, avg={train_nz_per_doc.mean():.1f}")
print(f"Val:   min={val_nz_per_doc.min()}, max={val_nz_per_doc.max()}, avg={val_nz_per_doc.mean():.1f}")

# Топ-10 самых частых слов
feature_names = bow_vectorizer.get_feature_names_out()
word_freq = np.array(X_train_bow.sum(axis=0)).flatten()
top_10_idx = np.argsort(word_freq)[::-1][:10]

print(f"\n4. ТОП-10 САМЫХ ЧАСТЫХ СЛОВ:")
for i, idx in enumerate(top_10_idx, 1):
    print(f"{i:2}. {feature_names[idx]:15} {word_freq[idx]:,}")

# Анализ первого документа
print(f"\n5. ПЕРВЫЙ ДОКУМЕНТ:")
first_doc = X_train_bow[0]
print(f"Ненулевых слов: {first_doc.nnz}")
if first_doc.nnz > 0:
    words = [feature_names[i] for i in first_doc.indices[:5]]
    values = first_doc.data[:5]
    print(f"Первые 5 слов: {list(zip(words, values))}")

# Визуализация
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

# Распределение частот слов
plt.subplot(1, 2, 1)
plt.hist(X_train_bow.data, bins=50, alpha=0.7, edgecolor='black')
plt.xlabel('Частота слова')
plt.ylabel('Количество')
plt.title('Распределение частот слов')
plt.grid(True, alpha=0.3)

# Топ-15 слов
plt.subplot(1, 2, 2)
top_15_idx = top_10_idx[:15] if len(top_10_idx) >= 15 else top_10_idx
top_words = [feature_names[i] for i in top_15_idx]
top_freqs = word_freq[top_15_idx]

plt.barh(range(len(top_words)), top_freqs[::-1])
plt.yticks(range(len(top_words)), top_words[::-1])
plt.xlabel('Частота')
plt.title('Топ-15 самых частых слов')

plt.tight_layout()
plt.show()

print("\n" + "="*50)
print("ВЫВОДЫ:")
print(f"• Матрицы очень разрежены ({train_density:.4f}% плотность)")
print(f"• В среднем {train_nz_per_doc.mean():.1f} слов на документ")
print(f"• Всего уникальных слов: {len(feature_names):,}")
print("="*50)

In [ ]:
# Анализ полученных признаков
def analyze_features(vectorizer, X_matrix, name):
    """Анализирует полученные признаки после векторизации"""
    print(f"\n=== АНАЛИЗ {name} ПРИЗНАКОВ ===")
    
    # Получаем частоты признаков
    feature_freq = np.array(X_matrix.sum(axis=0)).flatten()
    
    print(f"Общее количество признаков: {len(feature_freq)}")
    print(f"Разреженность матрицы: {1 - (X_matrix.nnz / (X_matrix.shape[0] * X_matrix.shape[1])):.3%}")
    print(f"Средняя частота признака: {feature_freq.mean():.2f}")
    print(f"Максимальная частота признака: {feature_freq.max()}")
    
    # Топ-20 самых частых признаков
    feature_names = vectorizer.get_feature_names_out()
    top_indices = feature_freq.argsort()[-20:][::-1]
    
    print(f"\nТоп-20 самых частых признаков:")
    for i, idx in enumerate(top_indices, 1):
        print(f"{i:2d}. {feature_names[idx]}: {int(feature_freq[idx])}")

# Анализ для TF-IDF
analyze_features(tfidf_vectorizer, X_train_tfidf, "TF-IDF")

# Анализ для BoW
analyze_features(bow_vectorizer, X_train_bow, "BAG-OF-WORDS")

In [ ]:
# Анализ n-грамм
def analyze_ngrams(vectorizer, name):
    """Анализирует распределение uni-grams и bi-grams"""
    feature_names = vectorizer.get_feature_names_out()
    
    unigrams = [f for f in feature_names if ' ' not in f]
    bigrams = [f for f in feature_names if ' ' in f]
    
    print(f"Uni-grams: {len(unigrams)} ({len(unigrams)/len(feature_names)*100:.1f}%)")
    print(f"Bi-grams: {len(bigrams)} ({len(bigrams)/len(feature_names)*100:.1f}%)")
    
    # Примеры bi-grams
    if bigrams:
        print(f"\nПримеры bi-grams (первые 10):")
        for bg in bigrams[:10]:
            print(f"  - {bg}")

analyze_ngrams(tfidf_vectorizer, "TF-IDF")
analyze_ngrams(bow_vectorizer, "BoW")

In [ ]:
# Анализ ключевых слов для основных меток (только для демонстрации)
def get_top_features_for_labels(vectorizer, X_matrix, y, top_n=10):
    """Находит самые характерные слова для основных меток"""
    # Получаем основные метки (первые метки из списков)
    primary_labels = [labels[0] if labels else 'none' for labels in y]
    
    # Берем топ-5 самых частых меток
    top_labels = pd.Series(primary_labels).value_counts().head(5).index
    
    feature_names = vectorizer.get_feature_names_out()   
 
    for label in top_labels:
        # Индексы документов с этой меткой
        label_indices = [i for i, lbls in enumerate(y) if label in lbls]
        
        if not label_indices:
            continue
            
        # Средний TF-IDF для этой метки
        label_vector = X_matrix[label_indices].mean(axis=0)
        label_scores = np.array(label_vector).flatten()
        
        # Топ-N признаков
        top_indices = label_scores.argsort()[-top_n:][::-1]
        
        print(f"\nМетка {label}:")
        for i, idx in enumerate(top_indices, 1):
            if label_scores[idx] > 0:
                print(f"  {i:2d}. {feature_names[idx]}: {label_scores[idx]:.4f}")

# Только для TF-IDF (так как у BoW нет весов)
get_top_features_for_labels(tfidf_vectorizer, X_train_tfidf, y_train)

## Применение простых моделей

### Шаг 1: Определение метрик качества

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# Ключевая метрика
key_metric = 'accuracy'

def evaluate_model_multilabel(y_true, y_pred, model_name):
    """Оценка модели для мульти-лейбл классификации"""
    # Accuracy для мульти-лейбл (доля полностью правильных предсказаний)
    accuracy = np.mean([set(true) == set(pred) for true, pred in zip(y_true, y_pred)])
    
    print(f"=== {model_name} ===")
    print(f"Accuracy: {accuracy:.4f}")
    
    return accuracy

# Обучение и оценка моделей для мульти-лейбл
def train_and_evaluate_models_multilabel(X_train, X_test, y_train, y_test):
    """Обучение и сравнение двух моделей для мульти-лейбл классификации"""
    
    # Модели с OneVsRest для мульти-лейбл
    knn = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5))
    lr = OneVsRestClassifier(LogisticRegression(random_state=42, max_iter=1000))
    
    # KNN
    knn.fit(X_train, y_train)
    y_pred_knn = knn.predict(X_test)
    acc_knn = evaluate_model_multilabel(y_test, y_pred_knn, "KNN")
    
    # Линейная модель
    lr.fit(X_train, y_train)
    y_pred_lr = lr.predict(X_test)
    acc_lr = evaluate_model_multilabel(y_test, y_pred_lr, "Logistic Regression")
    
    # Сравнение
    print("\n=== СРАВНЕНИЕ МОДЕЛЕЙ ===")
    print(f"KNN Accuracy: {acc_knn:.4f}")
    print(f"Logistic Regression Accuracy: {acc_lr:.4f}")
    
    if acc_knn > acc_lr:
        print("Лучшая модель: KNN")
    elif acc_lr > acc_knn:
        print("Лучшая модель: Logistic Regression")
    else:
        print("Модели показали одинаковую точность")
    
    return knn, lr, acc_knn, acc_lr


### Шаг 2: Baseline-модели

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import pandas as pd
import numpy as np

# Данные для обучения
vectorizers_data = {
    'TF-IDF': (X_train_tfidf, X_val_tfidf),
    'BOW': (X_train_bow, X_val_bow)
}

print("🚀 ОБУЧЕНИЕ МОДЕЛЕЙ")
print("=" * 30)

all_results = {}
mlb = MultiLabelBinarizer()

# Преобразуем метки в бинарный формат
y_train_bin = mlb.fit_transform(y_train)
y_val_bin = mlb.transform(y_val)

# Прогресс-бар для векторизаторов
for vec_name, (X_train_vec, X_val_vec) in tqdm(vectorizers_data.items(), desc="Векторизаторы"):
    print(f"\n📊 {vec_name}:")
    
    # Обучение моделей с прогресс-баром
    models = {
        'KNN': OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5)),
        'LogisticRegression': OneVsRestClassifier(LogisticRegression(random_state=42, max_iter=1000))
    }
    
    vec_results = {}
    
    for model_name, model in tqdm(models.items(), desc="Модели", leave=False):
        # Обучение
        model.fit(X_train_vec, y_train_bin)
        
        # Предсказание и оценка
        y_pred_bin = model.predict(X_val_vec)
        
        # Вычисляем accuracy для мульти-лейбл (доля полностью правильных предсказаний)
        accuracy = accuracy_score(y_val_bin, y_pred_bin)
        
        vec_results[model_name] = {'model': model, 'accuracy': accuracy}
        print(f"   {model_name}: {accuracy:.4f}")
    
    all_results[vec_name] = vec_results

# Итоговое сравнение
print("\n📈 ИТОГИ:")
results_list = []
for vec_name, models in all_results.items():
    for model_name, result in models.items():
        results_list.append({
            'Векторизатор': vec_name,
            'Модель': model_name,
            'Accuracy': result['accuracy']
        })

results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values('Accuracy', ascending=False)
print(results_df.to_string(index=False, float_format='%.4f'))

In [ ]:
#Функция для сравнения результатов
def compare_models(results_dict):
    """Сравнивает результаты всех моделей"""
    print("\n" + "="*60)
    print("СРАВНЕНИЕ МОДЕЛЕЙ")
    print("="*60)
    
    comparison_data = []
    for model_name, result in results_dict.items():
        metrics = result['metrics']
        comparison_data.append({
            'Model': model_name,
            'Accuracy': metrics['accuracy'],
            'F1 Macro': metrics['f1_macro'],
            'F1 Weighted': metrics['f1_weighted'],
            'F1 Micro': metrics['f1_micro'],
            'F1 Samples': metrics['f1_samples']
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df = comparison_df.sort_values('F1 Macro', ascending=False)
    
    print(comparison_df.to_string(index=False))
    
    return comparison_df

In [ ]:
#функция для анализа ошибок
def analyze_errors(y_true, y_pred, X_texts, top_n=10):
    """Анализирует наиболее частые ошибки"""
    errors = []
    
    for i, (true_labels, pred_labels, text) in enumerate(zip(y_true, y_pred, X_texts)):
        true_set = set(true_labels)
        pred_set = set(pred_labels)
        
        # Вычисляем различные типы ошибок
        false_positives = pred_set - true_set
        false_negatives = true_set - pred_set
        
        if false_positives or false_negatives:
            errors.append({
                'index': i,
                'text_preview': text[:100] + '...',
                'true_labels': true_labels,
                'pred_labels': pred_labels,
                'false_positives': list(false_positives),
                'false_negatives': list(false_negatives),
                'error_score': len(false_positives) + len(false_negatives)
            })
    
    errors_df = pd.DataFrame(errors)
    
    if len(errors_df) > 0:
        print(f"\n=== АНАЛИЗ ОШИБОК (ТОП-{top_n}) ===")
        top_errors = errors_df.nlargest(top_n, 'error_score')
        
        for _, error in top_errors.iterrows():
            print(f"\nОшибка #{error['index']} (счет: {error['error_score']}):")
            print(f"Текст: {error['text_preview']}")
            print(f"Истинные метки: {error['true_labels']}")
            print(f"Предсказанные метки: {error['pred_labels']}")
            if error['false_positives']:
                print(f"Ложные срабатывания: {error['false_positives']}")
            if error['false_negatives']:
                print(f"Пропущенные метки: {error['false_negatives']}")
            print("-" * 50)
    
    return errors_df

In [ ]:
def run_complete_pipeline(X_train, X_val, X_test, y_train, y_val, y_test, vectorizer_name):
    """Запускает полный пайплайн обучения и оценки моделей"""
    
    print(f"\n{'='*80}")
    print(f"ЗАПУСК ПАЙПЛАЙНА ДЛЯ {vectorizer_name}")
    print(f"{'='*80}")
    
    # Обучаем модели
    results = train_baseline_models(X_train, y_train, X_val, y_val, vectorizer_name)
    
    if not results:
        print("Ни одна модель не была успешно обучена!")
        return None
    
    # Сравниваем модели
    comparison_df = compare_models(results)
    
    # Выбираем лучшую модель
    best_model_name = comparison_df.iloc[0]['Model']
    best_result = results[best_model_name]
    
    print(f"\n🎯 ЛУЧШАЯ МОДЕЛЬ: {best_model_name}")
    print(f"📊 F1 Macro: {best_result['metrics']['f1_macro']:.4f}")
    
    # Анализ ошибок лучшей модели
    best_predictions = best_result['predictions']
    errors_df = analyze_errors(y_val, best_predictions, X_val, top_n=8)
    
    # Тестируем лучшую модель на тестовой выборке
    print(f"\n🧪 ТЕСТИРОВАНИЕ ЛУЧШЕЙ МОДЕЛИ НА ТЕСТОВОЙ ВЫБОРКЕ")
    
    # Преобразуем тестовые данные
    y_test_bin = best_result['mlb'].transform(y_test)
    y_test_pred_bin = best_result['model'].predict(X_test)
    y_test_pred = best_result['mlb'].inverse_transform(y_test_pred_bin)
    
    # Оценка на тестовой выборке
    test_metrics = evaluate_multilabel_model(y_test, y_test_pred, f"{best_model_name} - TEST SET")
    
    return {
        'best_model': best_result['model'],
        'best_model_name': best_model_name,
        'mlb': best_result['mlb'],
        'val_metrics': best_result['metrics'],
        'test_metrics': test_metrics,
        'all_results': results,
        'comparison': comparison_df
    }

### Шаг 3: Подбор гиперпараметров

In [ ]:
from sklearn.model_selection import GridSearchCV

print("=== ПОДБОР ГИПЕРПАРАМЕТРОВ ===")

# Оптимизация логистической регрессии
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2', 'none'],
    'class_weight': [None, 'balanced']
}

lr_grid = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=1000, multi_class='multinomial'),
    param_grid,
    cv=3,
    scoring='f1_macro',
    n_jobs=-1
)

print("Запуск GridSearch...")
lr_grid.fit(X_train_tfidf, y_train)

print(f"Лучшие параметры: {lr_grid.best_params_}")
print(f"Лучший F1-score на кросс-валидации: {lr_grid.best_score_:.4f}")

### Шаг 4: Анализ результатов

In [ ]:
# Тестирование лучшей модели
best_model = lr_grid.best_estimator_
y_pred_best = best_model.predict(X_val_tfidf)
best_metrics = evaluate_model(y_val, y_pred_best, "Best Logistic Regression")

# Сравнение всех моделей
print("\n=== СРАВНЕНИЕ МОДЕЛЕЙ ===")
models_comparison = pd.DataFrame({
    'KNN': knn_metrics,
    'LogisticRegression': lr_metrics,
    'NaiveBayes': nb_metrics,
    'BestModel': best_metrics
})

models_comparison

In [ ]:
# Визуализация сравнения моделей
plt.figure(figsize=(12, 6))

metrics_to_plot = ['accuracy', 'f1_macro', 'precision_macro', 'recall_macro']
models_comparison.T[metrics_to_plot].plot(kind='bar', figsize=(12, 6))
plt.title('Сравнение метрик моделей')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Матрица ошибок лучшей модели
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

plt.figure(figsize=(12, 10))
cm = confusion_matrix(y_val, y_pred_best, labels=best_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_model.classes_)
disp.plot(cmap='Blues', xticks_rotation=45, ax=plt.gca())
plt.title('Матрица ошибок - Best Logistic Regression')
plt.tight_layout()
plt.show()

In [ ]:
# Анализ самых частых ошибок
error_analysis = pd.DataFrame({
    'true': y_val,
    'predicted': y_pred_best,
    'text': X_val
})

errors = error_analysis[y_val != y_pred_best]
error_counts = errors.groupby(['true', 'predicted']).size().reset_index(name='count')
error_counts = error_counts.sort_values('count', ascending=False)

print("Самые частые ошибки классификации:")
print(error_counts.head(10))

print("\nПримеры ошибок:")
for i in range(min(3, len(errors))):
    row = errors.iloc[i]
    print(f"\nОшибка {i+1}:")
    print(f"Истинный класс: {row['true']}")
    print(f"Предсказанный класс: {row['predicted']}")
    print(f"Текст: {row['text'][:200]}...")

### Шаг 5: Финальные выводы

In [ ]:
print("=== ФИНАЛЬНЫЕ ВЫВОДЫ ===")

print(f"1. РАЗМЕТКА ДАННЫХ:")
print(f"   - Исходный датасет: {len(df)} записей")
print(f"   - После очистки и разметки: {len(df_labeled)} записей")
print(f"   - Количество классов: {df_labeled['target'].nunique()}")
print(f"   - Сбалансированность: медианный размер класса {class_sizes.median()}")

print(f"\n2. КАЧЕСТВО МОДЕЛЕЙ (F1-macro):")
print(f"   - KNN: {knn_metrics['f1_macro']:.4f}")
print(f"   - Naive Bayes: {nb_metrics['f1_macro']:.4f}")
print(f"   - Logistic Regression: {lr_metrics['f1_macro']:.4f}")
print(f"   - Best Model: {best_metrics['f1_macro']:.4f}")

print(f"\n3. РЕКОМЕНДАЦИИ:")
print(f"   - Лучшая модель: {type(best_model).__name__}")
print(f"   - Ключевая метрика: {key_metric}")
print(f"   - Основные проблемы: дисбаланс классов, семантически близкие классы")

# Сохранение лучшей модели
import joblib

model_artifacts = {
    'model': best_model,
    'vectorizer': tfidf_vectorizer,
    'metrics': best_metrics
}

joblib.dump(model_artifacts, 'best_model_artifacts.pkl')
print(f"\nЛучшая модель сохранена в 'best_model_artifacts.pkl'")